In [ ]:
import sys
sys.path.append('/home/ljq/code/MOO')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.interpolate import griddata
from PSO_func import *
from utils.inner_prod import *
from utils.Lumi_redshift import *  
from utils.PSD import *
from utils.zeropoint import *
from waveform.binary_waveform import *
from waveform.EMRI_waveform import *
from scipy.fftpack import fft, ifft,fftfreq
import math
from scipy.signal.windows import tukey
from config.config import config

In [ ]:
sys.path.append('/home/ljq/code/MOO/PSO')
from Sci_Obj.SNR_EMRI import *
from Sci_Obj.SNR_binary import *
from Sci_Obj.redshift_EMRI import *

In [5]:
def combined_func(x,PSD,**kwargs):
    return -SNR_binary_2(x,PSD)
#PSD=PSD_Sx
PSD=PSD_L_lambda
def crcbpso_test_func_mod(x,params):
    

    if len(x.shape) == 1:
        x=x.reshape(1,-1)
        n_row = 1
        fit_val=np.zeros(n_row)
        valid_pts=np.ones(n_row)
        valid_pts=crcb_chk_std_srch_rng(x)
        fit_val[~valid_pts]=np.inf
        x[valid_pts]=s2rv(x[valid_pts],params)

        for lpc in range(0,n_row):
            if valid_pts[lpc]:
                x_temp=x[lpc]
                fit_val[lpc]=combined_func(x_temp,PSD,figure_file=None)
    else:
        n_row, _ = x.shape
    
        fit_val=np.zeros(n_row)
        valid_pts=np.ones(n_row)
        valid_pts=crcb_chk_std_srch_rng(x)
        fit_val[~valid_pts]=np.inf

        x[valid_pts,:]=s2rv(x[valid_pts,:],params)

        for lpc in range(0,n_row):
            if valid_pts[lpc]:
                x_temp=x[lpc,:]
                fit_val[lpc]=combined_func(x_temp,PSD,figure_file=None)
    return fit_val,x,r2sv(x,params)

In [6]:
#Test crcbpso
#7个参数的取值范围
# ffparams = {
#    'r_min':np.array([5.0e6,25.0e6,59.0e6,57.0e6,3.363e9,0.44,2.06e-3,1.0*10**8,200.0]),
#    'r_max':np.array([5.0e6,25.0e6,59.0e6,57.0e6,3.363e9,0.44,2.06e-3,10.0*10**9,1200.0]) }

ffparams = {
   'r_min':np.array([1.0*10**8,200.0]),
   'r_max':np.array([10.0*10**9,1600.0]) }
#Fitness function handle
fit_func_handle=lambda x: crcbpso_test_func_mod(x,ffparams) 
#Call PSO
#np.random.seed(0)#Ensure that the random seed is correct
pso_out=crcbpso(fit_func_handle,2)
#Estimated parameters
std_coord=pso_out['best_location']
_,real_coord,_=fit_func_handle(std_coord)
print(f"Best location: {real_coord}")
print(f"Best fitness: {pso_out['best_fitness']}")

Step: 0 Best fitness -324083.74076583586
Step: 1 Best fitness -324425.42454891704
Step: 2 Best fitness -325602.1412080504
Step: 3 Best fitness -325602.1412080504
Step: 4 Best fitness -326048.5182197866
Step: 5 Best fitness -327566.1323578538
Step: 6 Best fitness -327711.5869717245
Step: 7 Best fitness -327711.5869717245
Step: 8 Best fitness -327711.5869717245
Step: 9 Best fitness -327711.5869717245
Step: 10 Best fitness -327711.5869717245
Step: 11 Best fitness -327711.5869717245
Step: 12 Best fitness -327711.5869717245
Step: 13 Best fitness -327711.5869717245
Step: 14 Best fitness -328037.1279779463
Step: 15 Best fitness -328037.1279779463
Step: 16 Best fitness -328092.17201868887
Step: 17 Best fitness -328092.17201868887
Step: 18 Best fitness -328092.17201868887
Step: 19 Best fitness -328092.17201868887
Step: 20 Best fitness -328092.17201868887
Step: 21 Best fitness -328092.17201868887
Step: 22 Best fitness -328092.17201868887
Step: 23 Best fitness -328092.17201868887
Step: 24 Best fi